In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

df = pd.read_csv('redone_data_analysis5.csv', index_col=0)
pd.options.display.max_colwidth = 100

In [2]:
# # Encoding school level as factors
# df['School Level'] = np.nan
# df.loc[df['Minimum Qual Requirements'].str.lower().str.contains('high school', na=False), 'School Level'] = 'High School'
# df.loc[df['Minimum Qual Requirements'].str.lower().str.contains('baccalaureate|college', na=False), 'School Level'] = "Bachelor's"
# df.loc[df['Minimum Qual Requirements'].str.lower().str.contains("master's", na=False), 'School Level'] = "Master's"
# df.loc[df['Minimum Qual Requirements'].str.lower().str.contains("professional engineer|registration as an architect", na=False), 'School Level'] = "Engineer/Architect License"
# df.loc[df['Minimum Qual Requirements'].str.lower().str.contains("state bar", na=False), 'School Level'] = "Bar"
# df.loc[df['Minimum Qual Requirements'].str.lower().str.contains("housing manager", na=False), 'School Level'] = "Housing Manager"

In [3]:
# # Converting all salaries to annual
# df.loc[df['Salary.Frequency'] == 'Hourly', 'Salary.Range.From'] = df['Salary.Range.From'] * 2080
# df.loc[df['Salary.Frequency'] == 'Hourly', 'Salary.Range.To'] = df['Salary.Range.To'] * 2080

In [5]:
lst = list(zip(df['Business.Title'], df['Preferred.Skills']))
vectorizer = TfidfVectorizer(stop_words = 'english', max_df = .8, min_df = 10)
td_matrix = vectorizer.fit_transform([str(i[1]) for i in lst])

In [6]:
td_matrix.shape
docs_compressed, s, words_compressed = svds(td_matrix, k=100)
words_compressed = words_compressed.T

In [71]:
docs_normed = normalize(docs_compressed, axis = 1, norm='l2')

In [72]:
columns = df.columns
dictionary = {i : col for i, col in enumerate(columns)}
new_dataframe = pd.concat([df, pd.DataFrame(docs_normed)], axis=1, ignore_index=True)
new_dataframe = new_dataframe.iloc[0:851]
new_dataframe.rename(dictionary, axis=1, inplace=True)

In [76]:
new_dataframe.drop(['Job.ID', 'Title.Code.No'], axis=1, inplace=True)

In [77]:
new_dataframe.to_csv('redone_data_analysis6.csv', index=False)